# Natural Language Processing
## Sentiment Analysis

### Imports

In [1]:
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
tf.__version__

'1.3.0'

### Load IMDB data set
    We are using the data set of aroung 50,000 reviews on movies from IMDB.

In [7]:
import Imdb